In [1]:
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
import os
from PIL import Image
import pandas as pd
from tqdm import tqdm

### get results

In [10]:
base_path  = '/scratch/hd2584/harmful-content-removal/data/images/baseline'
control_path  = '/scratch/hd2584/harmful-content-removal/data/images/control'
inpaint_path  = '/scratch/hd2584/harmful-content-removal/data/images/inpaint_new_mask_1219_revised_prompt'

img_id_ls = []
for root, dirs, files in os.walk(inpaint_path):
    for file in files:
        img_id_ls.append(file)

prompts_df = pd.read_csv("../data/prompts/lexica_hate_prompts.csv")

In [11]:
id_ls = []
cat_ls = []
sim_control_ls = []
sim_inpaint_ls = []

for img_id in tqdm(img_id_ls):
    img_id = img_id[:-4]
    imgae_base = cv2.imread(base_path + '/' + img_id + '.png', cv2.IMREAD_GRAYSCALE)
    imgae_contrl = cv2.imread(control_path + '/' + img_id + '.png', cv2.IMREAD_GRAYSCALE)
    imgae_inpaint = cv2.imread(inpaint_path + '/' + img_id + '.png', cv2.IMREAD_GRAYSCALE)
    imgae_inpaint = cv2.resize(imgae_inpaint, (imgae_base.shape[1], imgae_base.shape[0]))

    sim_control = ssim(imgae_base, imgae_contrl, full=True)[0]
    sim_control_ls.append(sim_control)

    sim_inpaint = ssim(imgae_base, imgae_inpaint, full=True)[0]
    sim_inpaint_ls.append(sim_inpaint)

    category = prompts_df.loc[prompts_df['unique_id'] == img_id, 'category'].iloc[0]
    cat_ls.append(category)

100%|██████████| 1147/1147 [02:33<00:00,  7.48it/s]


In [12]:
res_df = pd.DataFrame()
res_df['img_id'] = img_id_ls
res_df['category'] = cat_ls
res_df['sim_control'] = sim_control_ls
res_df['sim_inpaint'] = sim_inpaint_ls

In [13]:
pd.DataFrame(res_df.groupby('category')['sim_control'].mean())

,sim_control
category,
harassment,0.176686
hate,0.182768
illegal activity,0.162982
self-harm,0.248426
sexual,0.277751
shocking,0.224890
violence,0.202800


In [14]:
res_df['sim_control'].mean()

0.21524683292443644

In [15]:
pd.DataFrame(res_df.groupby('category')['sim_inpaint'].mean())

,sim_inpaint
category,
harassment,0.406830
hate,0.440278
illegal activity,0.429262
self-harm,0.452639
sexual,0.474685
shocking,0.430174
violence,0.396303


In [17]:
res_df['sim_inpaint'].mean()

0.43163307073791335

In [16]:
res_df.to_csv('similarity_scores.csv')